In [ ]:
import pandas as pd
from pycaret.classification import *
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cargar datos
df = pd.read_csv("caso3_rrhh_renuncias.csv")

# 2. Crear variable derivada para análisis: empleados satisfechos que igual se van
df['satisfecho_y_se_va'] = ((df['satisfaccion'] >= 4.0) & (df['renuncia'] == 1)).astype(int)

# 3. Visualizar si existen patrones ocultos
sns.boxplot(data=df, x='departamento', y='satisfaccion', hue='renuncia')
plt.title("Satisfacción y renuncias por departamento")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 4. Configuración avanzada de PyCaret
clf = setup(data=df,
            target='renuncia',
            session_id=2025,
            categorical_features=['departamento', 'tipo_contrato'],
            ignore_features=['satisfecho_y_se_va'],
            remove_multicollinearity=True,
            multicollinearity_threshold=0.85,
            feature_interaction=True,
            polynomial_features=True,
            bin_numeric_features=['antiguedad_meses', 'satisfaccion'],
            silent=True)

# 5. Seleccionar el mejor modelo con énfasis en evitar falsos negativos (Recall)
best = compare_models(sort='Recall')

# 6. Ajustar modelo y obtener curva de SHAP
final_model = tune_model(best, optimize='Recall')
interpret_model(final_model, plot='summary')  # SHAP summary plot

# 7. Crear matriz de decisiones para equipo de talento
preds = predict_model(final_model)
df_resultados = pd.concat([df.reset_index(drop=True), preds[['Label', 'Score']]], axis=1)

# 8. Filtrar empleados en riesgo con Score > 0.6 y Label = 1
foco_retencion = df_resultados[(df_resultados['Label'] == 1) & (df_resultados['Score'] > 0.6)]

# 9. Simular el impacto de una política de flexibilidad laboral
sim = df.copy()
sim.loc[(sim['tipo_contrato'] == 'Temporal') & (sim['home_office'] == 0), 'home_office'] = 1
sim_preds = predict_model(final_model, data=sim)

# 10. Comparar porcentaje de renuncias esperadas antes y después
antes = preds['Label'].mean()
despues = sim_preds['Label'].mean()
impacto = (antes - despues) * 100
print(f"Política simulada de trabajo remoto reduciría renuncias estimadas en: {impacto:.2f}%")

# 11. Guardar modelo final
save_model(final_model, 'modelo_retencion_talento')
